In [ ]:
#### Grabbing info from TJ's model explorations

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.formula.api import ols

df = pd.read_csv('data/kc_house_data.csv')

In [4]:
#This should be all data cleaning and pre-processimg
grade_num = []
for index, row in df.iterrows():
    grade_num.append(int((str(row['grade'])[0:2])))
df['grade_num'] = grade_num
df.drop('grade', axis = 1)
df['sqft_living_ratio'] = df['sqft_living'] / df['sqft_living15']
df['sqft_living_ratio'] = df['sqft_living_ratio'].loc[(np.abs(stats.zscore(df.sqft_living_ratio)) < 2)]
df = df.dropna(subset = ['sqft_living_ratio'])
df['sqft_lot_ratio'] = df['sqft_lot'] / df['sqft_lot15']
df['sqft_lot_ratio'] = df['sqft_lot_ratio'].loc[(np.abs(stats.zscore(df.sqft_lot_ratio)) < 2)]
df = df.dropna(subset = ['sqft_lot_ratio'])
df.waterfront.fillna(value='NO', inplace = True)
df.waterfront.replace( to_replace = ['NO','YES'], value =  [0,1], inplace = True)
df = df[df['bedrooms'] < 33]
df['sqft_basement'] = df['sqft_basement'].replace({'?': 0.0})
df['sqft_basement'] = df['sqft_basement'].astype(float)
df['view'] = df['view'].replace({'NONE': 0, 'AVERAGE': 1, 'GOOD': 2, 'FAIR': 3, 'EXCELLENT':4})
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit_transform(df[['zipcode']])
ord_enc = OrdinalEncoder()
ord_enc.fit_transform(df[['bedrooms','bathrooms','floors','grade_num']])



array([[ 2.,  2.,  0.,  3.],
       [ 2.,  7.,  2.,  3.],
       [ 3., 10.,  0.,  3.],
       ...,
       [ 1.,  1.,  2.,  3.],
       [ 2.,  8.,  2.,  4.],
       [ 1.,  1.,  2.,  3.]])

In [15]:
from sklearn.model_selection import train_test_split
X = df[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view',
       'sqft_above', 'sqft_basement', 'zipcode','sqft_living15', 'sqft_lot15', 'grade_num']]


X_train, X_test, y_train, y_test = train_test_split(X,df.price)

training_data = pd.concat([X_train,y_train], axis = 1)
testing_data = pd.concat([X_test,y_test], axis = 1)

In [16]:
from sklearn.linear_model import LinearRegression

baseline_model = LinearRegression()

In [17]:
from sklearn.model_selection import cross_validate, ShuffleSplit

most_correlated_feature = 'sqft_living'

splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)

baseline_scores = cross_validate(
    estimator=baseline_model,
    X=X_train[[most_correlated_feature]],
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Train score:     ", baseline_scores["train_score"].mean())
print("Validation score:", baseline_scores["test_score"].mean())

Train score:      0.47430713648383777
Validation score: 0.47209876179562954


In [18]:
ols('y_train ~ X_train', data = training_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                y_train   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     1780.
Date:                Tue, 16 Nov 2021   Prob (F-statistic):               0.00
Time:                        12:57:36   Log-Likelihood:            -2.0843e+05
No. Observations:               15243   AIC:                         4.169e+05
Df Residuals:                   15229   BIC:                         4.170e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -4.542e+07   3.39e+06    -13.385      0.000   -5.21e+07   -3.88e+07
X_train[0]  -2.775e+04   2529.348    -10.973      0.000   -3.27e+04   -2.28e+04
X_train[1]   -1.68e+04   3916.924     -4.289      0.000   -2.45e+04   -9123.253
X_train[2]    161.0867     23.963      6.722      0.000     114.116     208.057
X_train[3]     -0.1715      0.120     -1.431      0.153      -0.406       0.063
X_train[4]  -9816.2959   4326.188     -2.269      0.023   -1.83e+04   -1336.449
X_train[5]   5.377e+05   2.32e+04     23.144      0.000    4.92e+05    5.83e+05
X_train[6]   6.811e+04   2986.071     22.810      0.000    6.23e+04     7.4e+04
X_train[7]    -17.5643     23.810     -0.738      0.461     -64.234      29.106
X_train[8]     33.3806     23.669      1.410      0.158     -13.013      79.774
X_train[9]    458.1694     34.587     13.247      0.000     390.375     525.963
X_train[10]    56.2403      5.018     11.208      0.000      46.404      66.076
X_train[11]    -0.4282      0.135     -3.178      0.001      -0.692      -0.164
X_train[12]  9.569e+04   2643.353     36.202      0.000    9.05e+04    1.01e+05
==============================================================================
Omnibus:                     6784.774   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            79865.576
Skew:                           1.823   Prob(JB):                         0.00
Kurtosis:                      13.604   Cond. No.                     1.99e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.99e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
ohe = OneHotEncoder(sparse = False)
ohe.fit_transform(df[['zipcode']])
zip_encoded = ohe.fit_transform(df[['zipcode']])
zip_enc_df = pd.DataFrame(zip_encoded, columns=ohe.categories_[0], index=df.index)


df.drop("zipcode", axis=1, inplace=True)
df = pd.concat([df, zip_enc_df], axis=1)

In [32]:
ziplist = ohe.categories_[0].tolist()
zipliststr = []
for i in ziplist: 
    zipliststr.append(str(i))

ohe = OneHotEncoder(sparse = False)
ohe.fit_transform(df[['zipcode']])
zip_encoded = ohe.fit_transform(df[['zipcode']])
zip_enc_df = pd.DataFrame(zip_encoded, columns=ohe.categories_[0], index=df.index)


df.drop("zipcode", axis=1, inplace=True)
df = pd.concat([df, zip_enc_df], axis=1)zipliststr

['98001',
 '98002',
 '98003',
 '98004',
 '98005',
 '98006',
 '98007',
 '98008',
 '98010',
 '98011',
 '98014',
 '98019',
 '98022',
 '98023',
 '98024',
 '98027',
 '98028',
 '98029',
 '98030',
 '98031',
 '98032',
 '98033',
 '98034',
 '98038',
 '98039',
 '98040',
 '98042',
 '98045',
 '98052',
 '98053',
 '98055',
 '98056',
 '98058',
 '98059',
 '98065',
 '98070',
 '98072',
 '98074',
 '98075',
 '98077',
 '98092',
 '98102',
 '98103',
 '98105',
 '98106',
 '98107',
 '98108',
 '98109',
 '98112',
 '98115',
 '98116',
 '98117',
 '98118',
 '98119',
 '98122',
 '98125',
 '98126',
 '98133',
 '98136',
 '98144',
 '98146',
 '98148',
 '98155',
 '98166',
 '98168',
 '98177',
 '98178',
 '98188',
 '98198',
 '98199']

In [33]:
ohegrade = OneHotEncoder(sparse = False)
ohegrade.fit_transform(df[['grade']])
grade_encoded = ohe.fit_transform(df[['grade']])
grade_enc_df = pd.DataFrame(grade_encoded, columns=ohegrade.categories_[0], index=df.index)


df.drop("grade", axis=1, inplace=True)
df = pd.concat([df, grade_enc_df], axis=1)


gradelist = ohegrade.categories_[0].tolist()
gradeliststr = []
for i in gradelist: 
    gradeliststr.append(str(i))

#ord_enc = OrdinalEncoder()
#ord_enc.fit_transform(df[['bedrooms','bathrooms','floors','grade_num']])


In [34]:
df


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,10 Very Good,11 Excellent,12 Luxury,13 Mansion,4 Low,5 Fair,6 Low Average,7 Average,8 Good,9 Better
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,7237550310,5/12/2014,1230000.0,4,4.50,5420,101930,1.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
